[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/notebooks/agent10.ipynb)

# AGENT 10 SMELL (En construction)

# Objectifs

Démontrer la Smell Experiment

# Préparons les classes CompositeInteraction et Interaction

La classe `CompositeInteraction` est modifiée pour gérer l'enaction d'interactions composites de n'importe quel niveau hierarchique. 

On lui ajoute un attribut `_step` qui mémorise le pas courant de l'enaction en cours.

La méthode `current()` renvoie l'interaction primitive qui correspond au pas en cours.

La méthode `increment()` passe au pas suivant et renvoie l'interaction enactée si elle est terminée.

In [1]:
class CompositeInteraction:
    """A composite interaction is a tuple (pre_interaction, post_interaction) and a weight"""
    def __init__(self, pre_interaction, post_interaction):
        self.pre_interaction = pre_interaction
        self.post_interaction = post_interaction
        self.weight = 1
        self._step = 1

    def get_decision(self):
        """Return the flatten sequence of intermediary primitive interactions terminated with the final decision"""
        return f"{self.pre_interaction.sequence()}{self.post_interaction.get_decision()}"

    def get_actions(self):
        """Return the flat sequence of the decisions of this interaction as a string"""
        return f"{self.pre_interaction.get_actions()}{self.post_interaction.get_actions()}"
    
    def get_primitive_action(self):
        """Return the primite action"""
        return self.pre_interaction.get_primitive_action()

    def get_valence(self):
        """Return the valence of the pre_interaction plus the valence of the post_interaction"""
        return self.pre_interaction.get_valence() + self.post_interaction.get_valence()

    def reinforce(self):
        """Increment the composite interaction's weight"""
        self.weight += 1

    def key(self):
        """ The key to find this interaction in the dictionary is the string '<pre_interaction><post_interaction>'. """
        return f"({self.pre_interaction.key()},{self.post_interaction.key()})"

    def pre_key(self):
        """Return the key of the pre_interaction"""
        return self.pre_interaction.pre_key()

    def __str__(self):
        """ Print the interaction in the Newick tree format (pre_interaction, post_interaction: valence) """
        return f"({self.pre_interaction}, {self.post_interaction}: {self.weight})"

    def __eq__(self, other):
        """ Interactions are equal if they have the same pre and post interactions """
        if isinstance(other, self.__class__):
            return (self.pre_interaction == other.pre_interaction) and (self.post_interaction == other.post_interaction)
        else:
            return False

    def get_length(self):
        """Return the length of the number of primitive interactions in this composite interaction"""
        return self.pre_interaction.get_length() + self.post_interaction.get_length()

    def increment(self, interaction, interactions):
        """Increment the step of the appropriate sub-interaction. Return the enacted interaction if it is over, or None if it is ongoing."""
        # First step 
        if self._step == 1:
            interaction = self.pre_interaction.increment(interaction, interactions)
            # Ongoing pre-interaction. Return None
            if interaction is None:
                return None
            # Pre-interaction succeeded. Increment the step and return None
            elif interaction == self.pre_interaction:
                self._step = 2
                return None
            # Pre-interaction failed. Reset the step and return the enacted interaction
            else:
                self._step = 1
                return interaction
        # Second step
        else:
            interaction = self.post_interaction.increment(interaction, interactions)
            # Ongoing post-interaction. Return None
            if interaction is None:
                return None
            # Post-interaction succeeded. Reset the step and return this interaction
            elif interaction == self.post_interaction:
                self._step = 1
                return self
            # Post-interaction failed. Reset the step and return the enacted interaction
            else:
                self._step = 1
                composite_interaction = CompositeInteraction(self.pre_interaction, interaction)
                if composite_interaction.key() not in interactions:
                    # Add the enacted composite interaction to memory
                    interactions[composite_interaction.key()] = composite_interaction
                    print(f"Learning {composite_interaction}")
                    return composite_interaction
                else:
                    # Reinforce the existing composite interaction and return it
                    interactions[composite_interaction.key()].reinforce()
                    print(f"Reinforcing {interactions[composite_interaction.key()]}")
                    return interactions[composite_interaction.key()]

    def current(self):
        """Return the current intended primitive interaction"""
        # Step 1: the current primitive interaction of the pre-interaction
        if self._step == 1:
            return self.pre_interaction.current()
        # Step 2: The current primitive interaction of the post-interaction
        else:
            return self.post_interaction.current()

    def sequence(self):
        """Return the flat sequence of primitive interactions of this composite interaction"""
        return f"{self.pre_interaction.sequence()}{self.post_interaction.sequence()}"


On ajoute aussi les méthodes `current()` et `increment()` à la classe `Interaction` pour assurer la compatibilité avec la classe `CompositeInteraction`.

In [2]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, _action, _outcome, _valence):
        self._action = _action
        self._outcome = _outcome
        self._valence = _valence

    def get_action(self):
        """Return the action"""
        return self._action

    def get_actions(self):
        """Return the action as a string for compatibilty with CompositeInteraction"""
        return str(self._action)

    def get_decision(self):
        """Return the decision key"""
        return f"{self._action}"
        # return f"a{self._action}"

    def get_primitive_action(self):
        """Return the action for compatibility with CompositeInteraction"""
        return self._action

    def get_outcome(self):
        """Return the action"""
        return self._outcome

    def get_valence(self):
        """Return the action"""
        return self._valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self._action}{self._outcome}"

    def pre_key(self):
        """Return the key. Used for compatibility with CompositeInteraction"""
        return self.key()

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self._action}{self._outcome}:{self._valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        if isinstance(other, self.__class__):
            return self.key() == other.key()
        else:
            return False

    def get_length(self):
        """The length of the sequence of this interaction"""
        return 1

    def increment(self, interaction, interactions):
        """Return the enacted interaction for compatibility with composite interactions"""
        return interaction

    def current(self):
        """Return itself for compatibility with composite interactions"""
        return self

    def sequence(self):
        """Return the key. Use for compatibility with composite interactions"""
        return self.key()

# Implémentons l'agent 

In [3]:
import pandas as pd

class Agent:
    def __init__(self, _interactions):
        """ Initialize our agent """
        self._interactions = {interaction.key(): interaction for interaction in _interactions}
        self._primitive_intended_interaction = self._interactions["00"]
        self._intended_interaction = None

        # The context
        self._penultimate_interaction = None
        self._previous_interaction = None
        self._last_interaction = None
        self._penultimate_composite_interaction = None
        self._previous_composite_interaction = None
        self._last_composite_interaction = None
        
        # Prepare the dataframe of proposed interactions
        default_interactions = [interaction for interaction in _interactions if interaction.get_outcome() == 0]
        data = {'activated': [""] * len(default_interactions),
                'weight': [0] * len(default_interactions),
                'action': [i.get_action() for i in default_interactions],
                'actions': [i.get_actions() for i in default_interactions],
                'intention': [i.key() for i in default_interactions],
                'valence': [i.get_valence() for i in default_interactions],
                'decision': [i.get_decision() for i in default_interactions],
                'length': [1] * len(default_interactions),
                'primitive': [i.key() for i in default_interactions]}
        self._default_df = pd.DataFrame(data)
        self.proposed_df = None

    def action(self, _outcome):
        """Implement the agent's policy"""

        # Trace the previous cycle
        primitive_enacted_interaction = self._interactions[f"{self._primitive_intended_interaction.get_action()}{_outcome}"]
        print(
            f"Action: {self._primitive_intended_interaction.get_action()}, Prediction: {self._primitive_intended_interaction.get_outcome()}, "
            f"Outcome: {_outcome}, Prediction_correct: {self._primitive_intended_interaction.get_outcome() == _outcome}, "
            f"Valence: {primitive_enacted_interaction.get_valence()}")

        # Follow up the enaction
        if self._intended_interaction is None: # First interaction cycle
            enacted_interaction = primitive_enacted_interaction
        else:
            enacted_interaction = self._intended_interaction.increment(primitive_enacted_interaction, self._interactions)

        # If the intended interaction is over (completely enacted or aborted)
        if enacted_interaction is not None:
            # Memorize the context
            self._penultimate_composite_interaction = self._previous_composite_interaction
            self._previous_composite_interaction = self._last_composite_interaction
            self._penultimate_interaction = self._previous_interaction
            self._previous_interaction = self._last_interaction
            self._last_interaction = enacted_interaction
            # Call the learning mechanism
            self.learn(enacted_interaction)
            # Create the proposed dataframe
            self.create_proposed_df()
            self.aggregate_propositions()
            # Decide the next enaction
            self.decide()

        # Return the next primitive action
        self._primitive_intended_interaction = self._intended_interaction.current()
        return self._primitive_intended_interaction.get_action()
        
    def learn(self, enacted_interaction):
        """Learn the composite interactions"""
        # First level of composite interactions
        self._last_composite_interaction = self.learn_composite_interaction(self._previous_interaction, enacted_interaction)
        # Second level of composite interactions
        self.learn_composite_interaction(self._previous_composite_interaction, enacted_interaction)
        self.learn_composite_interaction(self._penultimate_interaction, self._last_composite_interaction)

        # Higher level composite interaction made of two composite interactions
        if self._last_composite_interaction is not None:
            self.learn_composite_interaction(self._penultimate_composite_interaction, self._last_composite_interaction)

    
    def learn_composite_interaction(self, pre_interaction, post_interaction):
        """Record or reinforce the composite interaction made of (pre_interaction, post_interaction)"""
        if pre_interaction is None:
            return None
        else:
            # If the pre-interaction exists
            composite_interaction = CompositeInteraction(pre_interaction, post_interaction)
            if composite_interaction.key() not in self._interactions:
                # Add the composite interaction to memory
                self._interactions[composite_interaction.key()] = composite_interaction
                print(f"Learning {composite_interaction}")
                return composite_interaction
            else:
                # Reinforce the existing composite interaction and return it
                self._interactions[composite_interaction.key()].reinforce()
                print(f"Reinforcing {self._interactions[composite_interaction.key()]}")
                return self._interactions[composite_interaction.key()]

    def create_proposed_df(self):
        """Create the proposed dataframe from the activated interactions"""
        # The list of activated interaction that match the current context
        activated_keys = [composite_interaction.key() for composite_interaction in
                          self._interactions.values()
                          if composite_interaction.get_length() > 1 and
                          (composite_interaction.pre_interaction == self._last_interaction or
                          composite_interaction.pre_interaction == self._last_composite_interaction)]
        data = {'activated': activated_keys,
                'weight': [self._interactions[k].weight for k in activated_keys],
                'action': [self._interactions[k].post_interaction.get_primitive_action() for k in activated_keys],
                'actions': [self._interactions[k].post_interaction.get_actions() for k in activated_keys],
                'intention': [self._interactions[k].post_interaction.key() for k in activated_keys],
                'valence': [self._interactions[k].post_interaction.get_valence() for k in activated_keys],
                'decision': [self._interactions[k].post_interaction.get_decision() for k in activated_keys],
                'primitive': [self._interactions[k].post_interaction.pre_key() for k in activated_keys],
                'length': [self._interactions[k].post_interaction.get_length() for k in activated_keys],
                }
        activated_df = pd.DataFrame(data).astype(self._default_df.dtypes)  # Force the same types for the case it is empty

        # Create the proposed dataframe
        self.proposed_df = pd.concat([self._default_df, activated_df], ignore_index=True)

        # Compute the proclivity for each proposition
        self.proposed_df['proclivity'] = self.proposed_df['weight'] * self.proposed_df['valence']

    def aggregate_propositions(self):
        """Aggregate the proclivity"""
        # Aggregate the proclivity for each decision
        grouped_df = self.proposed_df.groupby('decision').agg({'proclivity': 'sum', 'actions': 'first', 'action': 'first', 'length': 'first', 
                                                               'intention': 'first', 'primitive': 'first'}).reset_index()
        # For each proposed composite decision 
        for index, proposed in grouped_df[grouped_df['length'] > 1].iterrows():
            # print(f"Index {index}, actions {proposition['actions']}, intention {proposition['intention']}")
            # Find shorter decisions that start with the same sequence 
            for _, shorter in self.proposed_df[self.proposed_df.apply(lambda row: proposed['actions'].startswith(row['actions']) 
                                                                      and row['length'] < proposed['length'], axis=1)].iterrows():
                # Add the proclivity of the shorter decisions
                grouped_df.loc[index, 'proclivity'] += shorter['proclivity']
                # print(f"Decision {proposition['decision']} recieves {shorter['proclivity']} from partial {shorter['intention']}")
        
        # Sort by descending proclivity
        self.proposed_df = grouped_df.sort_values(by=['proclivity', 'decision'], ascending=[False, True]).reset_index(drop=True)

    def decide(self):
        """Selects the intended_interaction that at the top of the proposed dataframe"""
        # The intended interaction is in the first row because it has been sorted by descending proclivity
        intended_interaction_key = self.proposed_df.loc[0, 'intention']
        print("Intention:", intended_interaction_key)
        self._intended_interaction = self._interactions[intended_interaction_key]

# Implémentons l'environnement Smell 

In [379]:
save_dir = "sav"

FORWARD = 0
TURN_LEFT = 2
TURN_RIGHT = 1
FEEL_FRONT = 3
FEEL_LEFT = 4
FEEL_RIGHT = 5

STABLE = 0
BUMP = 1
INCREASE_LEFT = 2
INCREASE_RIGHT = 3
INCREASE_FRONT = 4
DECREASE = 5
EAT = 6

agent_color = "#1976D2"
colors = ["#D6D6D6", '#5C946E', '#FAE2DB', '#535865', "#F93943", "#E365C1"]

BUMPING = 4
TARGET = 5

On crée l'environnement Smell

In [776]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

# Directions
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

class SmallLoop():
    def __init__(self, position, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.position = np.array(position)  # Using NumPy array of shape (2)
        self.direction = direction
        self.cmap = ListedColormap(colors)
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5], self.cmap.N)
        self.marker_size = 400
        self.marker_map = {LEFT: '<', DOWN: 'v', RIGHT: '>', UP: '^'}
        self.marker_color = agent_color
        self.directions = np.array([
            [0, -1],  # Left
            [1, 0],   # Down
            [0, 1],   # Right
            [-1, 0]   # Up
        ])
        self.previous_smell = 0
        self.smell_feedback = np.array([
            [STABLE,   INCREASE_LEFT, INCREASE_RIGHT, INCREASE_FRONT], 
            [DECREASE, INCREASE_LEFT, STABLE,         INCREASE_FRONT], 
            [DECREASE, STABLE,        INCREASE_RIGHT, INCREASE_FRONT], 
            [DECREASE, DECREASE,      DECREASE,       INCREASE_FRONT] 
        ])
        

    def smell(self):
        """Return the smell feedback: 1: smell left, 2: smell right, 3: smell front"""
        smell_up_left = np.any(self.grid[:self.position[0]+1, :self.position[1]+1] == TARGET) 
        smell_up_right = np.any(self.grid[:self.position[0]+1, self.position[1]:] == TARGET) 
        smell_down_left = np.any(self.grid[self.position[0]:, :self.position[1]+1] == TARGET) 
        smell_down_right = np.any(self.grid[self.position[0]:, self.position[1]:] == TARGET) 
        print(f"Smell up-left:{smell_up_left}, up-right:{smell_up_right}, down-left:{smell_down_left}, down-right:{smell_down_right}")
        smell_left = {LEFT: smell_down_left, DOWN: smell_down_right, RIGHT: smell_up_right, UP: smell_up_left}[self.direction]
        smell_right = {LEFT: smell_up_left, DOWN: smell_down_left, RIGHT: smell_down_right, UP: smell_up_right}[self.direction]
        smell = smell_left + 2 * smell_right 
        result = self.smell_feedback[self.previous_smell, smell]
        self.previous_smell = smell
        return result
    
    def outcome(self, action):
        """Update the grid. Return the outcome of the action."""
        result = STABLE

        if action == FORWARD:  
            target_position = self.position + self.directions[self.direction]
            if self.grid[tuple(target_position)] in [0, TARGET]:  # Don't bump in targets
                self.position[:] = target_position
            else:
                result = BUMP
                self.maze[tuple(target_position)] = BUMPING
        
        elif action == TURN_RIGHT:
            self.direction = {LEFT: UP, DOWN: LEFT, RIGHT: DOWN, UP: RIGHT}[self.direction]
        
        elif action == TURN_LEFT:
            self.direction = {LEFT: DOWN, DOWN: RIGHT, RIGHT: UP, UP: LEFT}[self.direction]
        
        elif action == FEEL_FRONT:
            feeling_position = self.position + self.directions[self.direction]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_LEFT:
            feeling_position = self.position + self.directions[(self.direction + 1) % 4]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL
        
        elif action == FEEL_RIGHT:
            feeling_position = self.position + self.directions[self.direction - 1]
            if self.grid[tuple(feeling_position)] == 0:
                self.maze[tuple(feeling_position)] = FEELING_EMPTY
            else:
                result = BUMP
                self.maze[tuple(feeling_position)] = FEELING_WALL

        # Smell
        smell_feedback = self.smell()
        if self.grid[self.position[0], self.position[1]] == TARGET:
            self.grid[self.position[0], self.position[1]] = 0
            self.maze[self.position[0], self.position[1]] = 0
            result = EAT
        # If not bump then smell_feeback
        if not result:
            result = smell_feedback
        
        print(f"Line: {self.position[0]}, Column: {self.position[1]}, direction: {self.direction}")
        return result  
    
    def display(self):
        """Display the grid in the notebook"""
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
            plt.show()
    
    def save(self, step):
        """Save the display as a PNG file"""
        fig, ax = plt.subplots()
        ax.set_xticks([])
        ax.set_yticks([])
        ax.axis('off')
        ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
        plt.scatter(self.position[1], self.position[0], s=self.marker_size, marker=self.marker_map[self.direction], c=self.marker_color)
        ax.text(4.5, 0, f"{step:>3}", fontsize=12, color='White')
        plt.savefig(f"{save_dir}/{step:03}.png", bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close(fig)
    
    def clear(self, clear):
        """Clear the grid display"""
        if clear:
            self.maze[:, :] = self.grid


# Testons l'agent avec 3 actions

In [1243]:
# Instanciate the small loop environment
e = SmallLoop([6, 4], UP)

# Instanciate the agent 
interactions = [
    Interaction(FORWARD, STABLE, -1),
    Interaction(FORWARD, BUMP,-5),
    Interaction(FORWARD, INCREASE_LEFT, 5),
    Interaction(FORWARD, INCREASE_RIGHT, 5),
    Interaction(FORWARD, INCREASE_FRONT, 10),
    Interaction(FORWARD, DECREASE, -10),
    Interaction(FORWARD, EAT, 0),
    Interaction(TURN_LEFT, STABLE, 0),
    Interaction(TURN_LEFT, INCREASE_LEFT, 10),
    Interaction(TURN_LEFT, INCREASE_RIGHT, 10),
    Interaction(TURN_LEFT, INCREASE_FRONT, 15),
    Interaction(TURN_LEFT, DECREASE, -10),
    Interaction(TURN_RIGHT, STABLE, 0),
    Interaction(TURN_RIGHT, INCREASE_LEFT, 10),
    Interaction(TURN_RIGHT, INCREASE_RIGHT, 10),
    Interaction(TURN_RIGHT, INCREASE_FRONT, 15),
    Interaction(TURN_RIGHT, DECREASE, -10),
]
a = Agent(interactions)

# Run the interaction loop
step = 0
outcome = 0

# Display
out = Output()
e.display()
display(out)

Output()

## En mode pas à pas

In [1268]:
print(f"Step: {step}")
step += 1
action = a.action(outcome)
e.display()
e.clear(True)
# e.save(step)  # Sauvegarde le fichier image qui servira au gif
outcome = e.outcome(action)
a.proposed_df

Step: 24
Action: 0, Prediction: 0, Outcome: 6, Prediction_correct: False, Valence: 0
Learning (04:10, 06:0: 1)
Learning ((14:15, 04:10: 1), 06:0: 1)
Learning (14:15, (04:10, 06:0: 1): 1)
Learning (((13:10, 03:5: 2), 14:15: 1), (04:10, 06:0: 1): 1)
Intention: 00
Smell up-left:False, up-right:False, down-left:False, down-right:False
Line: 4, Column: 6, direction: 1


,decision,proclivity,actions,action,length,intention,primitive
0,0,0,0,0,1,00,00
1,1,0,1,1,1,10,10
2,2,0,2,2,1,20,20


Insérons une nouvelle cible

In [863]:
e.grid[3,3] = TARGET
e.maze[3,3] = TARGET
e.display()

In [1206]:
e.grid[6, 5] = TARGET
e.maze[6, 5] = TARGET
e.display()

In [1223]:
e.grid[2, 9] = TARGET
e.maze[2, 9] = TARGET
e.display()

# Créons le film gif

In [4666]:
import imageio.v2 as imageio
import os

img_dir = f"./{save_dir}"
all_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.png')]
images = [imageio.imread(f) for f in all_files]
imageio.mimsave("movie.gif", images, fps=4)